# Actividad Integradora 1

 Mobilidad Urbana 

In [ ]:

import agentpy as ap
import pathfinding as pf 
import matplotlib.pyplot as plt 
from owlready2 import *  
import itertools
import IPython
import math

# Creacion de Ontologia



In [3]:
onto = get_ontology("file://content/coin_onto.owl")

with onto:
    
    class Entity(Thing):
        pass
    
    class Hunter(Entity):
        pass
    
    class Coin(Entity):
        pass
    
    class Place(Thing):
        pass
    
    
    #Propiedad para describir el lugar de la entidad en el grid
    
    class is_in_place(ObjectProperty):
        domain = [Entity]
        range = [Place]
        pass
    
    #Propiedad para especificar la posicion de los lugares
    
    class at_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]
        pass
    
    #Propiedad para decir cuantas monedas puede ver el agente
    
    class coins_within_reach(ObjectProperty):
        domain = [Hunter]
        range = [int]
        pass

# Clase para el agente de Camion levantador de cajas
## BDI

Agente con racionamiento practico de tipo BDI

In [4]:
class HunterAgent(ap.Agent):
    
    #perception aka see()
    def see(self,e):
        #aqui viene el rango de vision del levanta carga
        #en caso de querer percepcion completa del mundo seria
        #seeRange = self.model.p.worldSize[0]
        seeRange = 10
        #vecimos osea cajas
        p = [a for a in e.neighbors(self, distance=seeRange) if a.agentType==1 ]
        return p
    
    #Belife Revision Function BRF
    #esta basado en la ontologia
    def brf(self, p):
        
        #destruye los beliefs anteriores
        for coin in self.this_hunter.coins_within_reach:
            destroy_entity(coin.is_in_place[0])
            destroy_entity(coin)
        destroy_entity(self.this_hunter.is_in_place[0])
        
        #inicializa el hunter ontologicamente
        currentPos = self.model.coinWorld.positions[self]
        self.this_hunter.is_in_place = [Place(at_position = str(currentPos))]
        
        #inicializa la caja a la vista ontologicamente
        for c in p:
            theCoin = Coin(is_in_place = [Place()])
            theCoin.is_in_place[0].at_position = str(self.model.coinWorld.positions[c])
            self.this_hunter.coins_within_reach.append(theCoin)
            
    #Opt funcition, de aqui se reciben los Desires 
    def options(self):
        
        distance = {}
        
        #por cada caja que el hunter ve
        for onto_coin in self.this_hunter.coins_within_reach:
            coin_pos = eval(onto_coin.is_in_place[0].at_position)
        